# European Soccer Database - Data Exploration

This notebook explores the Kaggle European Soccer Database to understand its structure, available tables, and data quality before building the clustering pipeline.

## Objectives
1. Connect to SQLite database
2. Inspect available tables and their schemas
3. Explore match data structure
4. Check data quality and completeness
5. Identify relevant columns for feature engineering

In [ ]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 100)

# Path to database (update if needed)
DB_PATH = "../data/database.sqlite"

## 1. Connect to Database and Inspect Tables

In [ ]:
# Connect to database
conn = sqlite3.connect(DB_PATH)

# Get all table names
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
table_names = [table[0] for table in tables]

print("Available tables in database:")
for table in table_names:
    print(f"  - {table}")

## 2. Examine Table Schemas

In [ ]:
# Examine schema for each table
for table_name in table_names:
    print(f"\n{'='*60}")
    print(f"Table: {table_name}")
    print(f"{'='*60}")
    cursor.execute(f"PRAGMA table_info({table_name})")
    columns = cursor.fetchall()
    for col in columns:
        print(f"  {col[1]:<30} {col[2]:<15} Nullable: {not col[3]}")

## 3. Explore Match Data

In [ ]:
# Load match data
query = "SELECT * FROM Match LIMIT 5"
matches_sample = pd.read_sql_query(query, conn)
print("Sample match data:")
matches_sample.head()

In [ ]:
# Get full match data shape and info
query = "SELECT * FROM Match"
matches_full = pd.read_sql_query(query, conn)
print(f"Total matches: {len(matches_full)}")
print(f"\nColumns in Match table: {list(matches_full.columns)}")
print(f"\nData types:\n{matches_full.dtypes}")

## 4. Check for Top 5 Leagues

In [ ]:
# Check league information
query = """
SELECT l.id, l.name, c.name as country_name
FROM League l
JOIN Country c ON l.country_id = c.id
ORDER BY l.id
"""
leagues = pd.read_sql_query(query, conn)
print("Available leagues:")
print(leagues)

In [ ]:
# Check match count by league
query = """
SELECT l.name as league_name, COUNT(*) as match_count
FROM Match m
JOIN League l ON m.league_id = l.id
GROUP BY l.name
ORDER BY match_count DESC
"""
league_counts = pd.read_sql_query(query, conn)
print("\nMatch counts by league:")
print(league_counts)

## 5. Examine Match Statistics Columns

The Match table likely contains detailed statistics stored in JSON or separate columns. Let's check what's available.

In [ ]:
# Check for statistical columns (shots, possession, etc.)
# Common column patterns to look for
stat_columns = [col for col in matches_full.columns 
                if any(term in col.lower() for term in 
                       ['shot', 'possession', 'pass', 'tackle', 'foul', 'card', 'corner'])]

print(f"Statistical columns found: {len(stat_columns)}")
print("\nStatistical columns:")
for col in stat_columns[:20]:  # Show first 20
    print(f"  - {col}")

if len(stat_columns) > 20:
    print(f"\n... and {len(stat_columns) - 20} more")

## 6. Check Data Quality

Examine missing values and data completeness for key columns.

In [ ]:
# Check missing values
missing_data = matches_full.isnull().sum()
missing_percent = (missing_data / len(matches_full)) * 100

missing_df = pd.DataFrame({
    'Column': missing_data.index,
    'Missing Count': missing_data.values,
    'Missing %': missing_percent.values
})
missing_df = missing_df[missing_df['Missing Count'] > 0].sort_values('Missing Count', ascending=False)

print(f"Columns with missing data: {len(missing_df)}")
print("\nTop columns with missing data:")
missing_df.head(20)

## 7. Examine Team and Season Information

In [ ]:
# Check team data
query = "SELECT * FROM Team LIMIT 10"
teams_sample = pd.read_sql_query(query, conn)
print("Sample team data:")
teams_sample.head()

In [ ]:
# Check available seasons
if 'season' in matches_full.columns:
    seasons = matches_full['season'].unique()
    print(f"Available seasons: {sorted(seasons)}")
    print(f"\nNumber of matches per season:")
    print(matches_full['season'].value_counts().sort_index())
elif 'date' in matches_full.columns:
    print("No 'season' column found. Checking date column...")
    print(f"Date range: {matches_full['date'].min()} to {matches_full['date'].max()}")

## 8. Summary and Next Steps

### Key Findings:
- [Fill in after exploration]

### Columns Identified for Feature Engineering:
- **Attack**: [e.g., home_team_goal, away_team_goal, shoton, shotoff]
- **Possession**: [e.g., possession columns if available]
- **Defense**: [e.g., goals conceded]
- **Game State**: [e.g., performance in different match situations]

### Next Steps:
1. Finalize feature extraction strategy based on available columns
2. Implement data loading functions in `src/data_loader.py`
3. Build feature engineering pipeline in `src/feature_engineering.py`

In [ ]:
# Close database connection
conn.close()
print("Database connection closed.")